# RAG met Azure AI Agent Service & Semantic Kernel

Deze codevoorbeeld laat zien hoe je een Azure AI-agent kunt maken en beheren voor retrieval-augmented generation (RAG) met behulp van de `Azure AI Agent Service` en `Semantic Kernel`. De agent verwerkt gebruikersvragen op basis van de opgehaalde context en geeft dienovereenkomstig nauwkeurige antwoorden.


SQLite-versie oplossen  
Als je de foutmelding tegenkomt:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Haal dit codeblok aan het begin van je notebook uit commentaar:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Pakketten importeren
De volgende code importeert de benodigde pakketten:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation met Semantic Kernel & Azure AI Agent Service

Dit voorbeeld laat zien hoe je de **Azure AI Agent Service** kunt gebruiken om **Retrieval-Augmented Generation (RAG)** uit te voeren door een taalmodel te combineren met domeinspecifieke context uit een geüpload document.

### Hoe Het Werkt

1. **Document Upload**: Een markdown-bestand (document.md) met informatie (Contoso's reisverzekeringsbeleid) wordt geüpload naar de agent service.

2. **Vector Store Creatie**: Het document wordt geïndexeerd in een vector store om semantisch zoeken in de inhoud mogelijk te maken.

3. **Agent Configuratie**: Een agent wordt ingesteld met het `gpt-4o` model en de volgende strikte instructies:
   - Beantwoord alleen vragen op basis van opgehaalde inhoud uit het document.
   - Weiger te antwoorden als de vraag buiten de scope valt.

4. **File Search Tool Integratie**: De `FileSearchTool` wordt geregistreerd bij de agent, waardoor het model relevante fragmenten uit het geïndexeerde document kan zoeken en ophalen tijdens inferentie.

5. **Gebruikersinteractie**: Gebruikers kunnen vragen stellen. Als relevante informatie in het document wordt gevonden, genereert de agent een onderbouwd antwoord.  
   Als dat niet het geval is, geeft de agent expliciet aan dat het document niet voldoende informatie bevat.


### Hoofdfunctie

```python
# Dit is de hoofdfunctie van het programma
def main():
    print("Welkom bij het programma!")  # Begroet de gebruiker
    resultaat = bereken_som(5, 10)  # Roept de functie aan om een som te berekenen
    print(f"De som is: {resultaat}")  # Toont het resultaat aan de gebruiker
```

### Bereken Som

```python
# Deze functie berekent de som van twee getallen
def bereken_som(a, b):
    return a + b  # Geeft de som van a en b terug
```

### Uitvoeren van het Programma

```python
# Controleert of dit script direct wordt uitgevoerd
if __name__ == "__main__":
    main()  # Voert de hoofdfunctie uit
```


Zorg ervoor dat u eerst `az login` uitvoert met behulp van de Azure CLI, zodat de juiste authenticatiecontext wordt geboden bij het gebruik van de `DefaultAzureCredential`. De Azure AI Agent Service maakt geen gebruik van API-sleutels.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Got it! Please provide the markdown file you'd like me to translate.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
